In [1]:
import tensorflow as tf

import numpy as np
import os
import time

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')


1122304/1115394 [==============================] - 1s 1us/step


In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [4]:
path_to_file

'C:\\Users\\hayashi\\.keras\\datasets\\shakespeare.txt'

In [5]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [10]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

65 unique characters


In [11]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [15]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '&' :   4,
  "'" :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '3' :   9,
  ':' :  10,
  ';' :  11,
  '?' :  12,
  'A' :  13,
  'B' :  14,
  'C' :  15,
  'D' :  16,
  'E' :  17,
  'F' :  18,
  'G' :  19,
  ...
}


In [16]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'First Citizen' ---- characters mapped to int ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [17]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

In [20]:
print( len(text), examples_per_epoch)


1115394 11043


In [22]:
# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [23]:
for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

F
i
r
s
t


In [24]:
text_as_int.shape

(1115394,)

In [26]:
seq_length= 4
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'First'
' Citi'
'zen:\n'
'Befor'
'e we '


In [47]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [51]:
for input_example, target_example in  dataset.take(2):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Firs'
Target data: 'irst'
Input data:  ' Cit'
Target data: 'Citi'


In [50]:
for input_example, target_example in  dataset.take(2):
  print ('Input data: ', input_example.numpy())
  print ('Target data:', target_example.numpy())

Input data:  [18 47 56 57]
Target data: [47 56 57 58]
Input data:  [ 1 15 47 58]
Target data: [15 47 58 47]


In [43]:
dataset.take(1)

<TakeDataset shapes: ((64, 4), (64, 4)), types: (tf.int32, tf.int32)>

In [53]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 1 (' ')
  expected output: 15 ('C')
Step    1
  input: 15 ('C')
  expected output: 47 ('i')
Step    2
  input: 47 ('i')
  expected output: 58 ('t')
Step    3
  input: 58 ('t')
  expected output: 47 ('i')


In [54]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 4), (64, 4)), types: (tf.int32, tf.int32)>

In [55]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [56]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [57]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [59]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 4, 65) # (batch_size, sequence_length, vocab_size)


In [60]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (64, None, 256)           16640     
_________________________________________________________________
gru_1 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
dense_1 (Dense)              (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [61]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [63]:
s= tf.random.categorical(example_batch_predictions[0], num_samples=1)
s

<tf.Tensor: shape=(4, 1), dtype=int64, numpy=
array([[47],
       [21],
       [11],
       [22]], dtype=int64)>

In [65]:
ss = tf.squeeze(s,axis=-1).numpy()
ss

array([47, 21, 11, 22], dtype=int64)

In [66]:
sampled_indices

array([61, 37, 13, 15], dtype=int64)

In [67]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 4, 65)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.174882


In [68]:
model.compile(optimizer='adam', loss=loss)

In [69]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [70]:
EPOCHS=10

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
3485/3485 [==============================] - 270s 77ms/step - loss: 2.2192
Epoch 2/10
3485/3485 [==============================] - 290s 83ms/step - loss: 2.1012
Epoch 3/10
3485/3485 [==============================] - 294s 84ms/step - loss: 2.0861
Epoch 4/10
3485/3485 [==============================] - 294s 84ms/step - loss: 2.0823
Epoch 5/10
3485/3485 [==============================] - 300s 86ms/step - loss: 2.0817
Epoch 6/10
3485/3485 [==============================] - 311s 89ms/step - loss: 2.0821
Epoch 7/10
3485/3485 [==============================] - 323s 93ms/step - loss: 2.0838
Epoch 8/10
3485/3485 [==============================] - 314s 90ms/step - loss: 2.0857
Epoch 9/10
1820/3485 [==============>...............] - ETA: 2:27 - loss: 2.083